# Streamlit example development tuning

In this notebook, we load a few configurations of the streamlit core app and run a test set through TruLens evaluations. This shows how to test and find the optimal configuration of any LLM app, carefully keeping track of configuration metadata.

In [ ]:
import os
from dotenv import load_dotenv
from common_ui import TruCustomApp, generator, ModelConfig, feedbacks_rag
import streamlit as st

from trulens_eval import Tru

load_dotenv()

db_url = 'snowflake://{user}:{password}@{account}/{dbname}/{schema}?warehouse={warehouse}&role={role}'.format(
        user=os.environ['SF_USER'],
        account=os.environ['SF_ACCOUNT'],
        password=os.environ['SF_PASSWORD'],
        dbname=os.environ['SF_DB_NAME'],
        schema=os.environ['SF_SCHEMA'],
        warehouse=os.environ['SF_WAREHOUSE'],
        role=os.environ['SF_ROLE']
    )

tru = Tru(database_url=db_url)

## Create the test set

In [ ]:
from schema import Conversation, Message

def create_conversation_from_user_message(user_message: str) -> Conversation:
    conversation = Conversation()
    conversation.add_message(Message(role="user", content=user_message))
    return conversation

In [ ]:
prompts = [
    "How do I launch a streamlit app?",
    "How can I capture the state of my session in streamlit?",
    "How do I install streamlit?",
    "How do I change the background color of a streamlit app?",
    "What's the advantage of using a streamlit form?",
    "What are some ways I should use checkboxes?",
    "How can I conserve space and hide away content?",
    "Can you recommend some resources for learning Streamlit?",
    "What are some common use cases for Streamlit?",
    "How can I deploy a streamlit app on the cloud?",
    "How do I add a logo to streamlit?",
    "What is the best way to deploy a Streamlit app?",
    "How should I use a streamlit toggle?",
    "How do I add new pages to my streamlit app?",
    "How do I write a dataframe to display in my dashboard?",
    "Can I plot a map in streamlit? If so, how?",
    "How do vector stores enable efficient similarity search?",
    "How do I prevent my child from using the internet?",
    "What should I pack for a camping trip this weekend?",
    "How do I defend myself against bear attacks?"
]

conversations = [create_conversation_from_user_message(prompt) for prompt in prompts]


## Test the first app

In [ ]:

tru_recorder_1 = TruCustomApp(generator, app_id="app-dev-1-pinecone", metadata={"model": "Snowflake Arctic", "use_rag": True, "temperature": 0.2, "top_p": 0.1, "retriever": "pinecone"},  feedbacks=feedbacks_rag)

with tru_recorder_1:
    for conversation in conversations:
        model_config_1 = ModelConfig(model="Snowflake Arctic", use_rag=True, temperature=0.2, top_p=0.1, retriever="Pinecone", retrieval_filter=0)
        chat = st.chat_message("assistant")
        conversation.model_config = model_config_1
        user_message, prompt = generator.prepare_prompt(conversation)
        if conversation.model_config.use_rag:
            text_response = generator.retrieve_and_generate_response(user_message, prompt, conversation, chat)

In [ ]:

tru_recorder_2 = TruCustomApp(generator, app_id="app-dev-2-cortex-search", metadata={"model": "Snowflake Arctic", "use_rag": True, "temperature": 0.2, "top_p": 0.1, "retriever": "cortex"},  feedbacks=feedbacks_rag)

with tru_recorder_2:
    for conversation in conversations:
        model_config_2 = ModelConfig(model="Snowflake Arctic", use_rag=True, temperature=0.2, top_p=0.1, retriever="Cortex Search", retrieval_filter=0)
        chat = st.chat_message("assistant")
        conversation.model_config = model_config_2
        user_message, prompt = generator.prepare_prompt(conversation)
        if conversation.model_config.use_rag:
            text_response = generator.retrieve_and_generate_response(user_message, prompt, conversation, chat)

In [ ]:

tru_recorder_3 = TruCustomApp(generator, app_id="app-dev-3-cortex-search-w-filters", metadata={"model": "Snowflake Arctic", "use_rag": True, "temperature": 0.2, "top_p": 0.1, "retriever": "cortex", "retrieval_filter":0.75, "filter_feedback_function": "Context Relevance (LLM-as-Judge)"},  feedbacks=feedbacks_rag)

with tru_recorder_3:
    for conversation in conversations:
        model_config_3 = ModelConfig(model="Snowflake Arctic", use_rag=True, temperature=0.2, top_p=0.1, retriever="Cortex Search", retrieval_filter=0.75, filter_feedback_function="Context Relevance (LLM-as-Judge)")
        chat = st.chat_message("assistant")
        conversation.model_config = model_config_3
        user_message, prompt = generator.prepare_prompt(conversation)
        if conversation.model_config.use_rag:
            text_response = generator.retrieve_and_generate_response(user_message, prompt, conversation, chat)

In [ ]:

tru_recorder_4 = TruCustomApp(generator, app_id="app-dev-4-cortex-search-w-filters-small_model", metadata={"model": "Snowflake Arctic", "use_rag": True, "temperature": 0.2, "top_p": 0.1, "retriever": "cortex", "retrieval_filter":0.75, "filter_feedback_function": "Context Relevance (small)"},  feedbacks=feedbacks_rag)

with tru_recorder_4:
    for conversation in conversations:
        model_config_4= ModelConfig(model="Snowflake Arctic", use_rag=True, temperature=0.2, top_p=0.1, retriever="Cortex Search", retrieval_filter=0.75, filter_feedback_function="Context Relevance (small)")
        chat = st.chat_message("assistant")
        conversation.model_config = model_config_4
        user_message, prompt = generator.prepare_prompt(conversation)
        if conversation.model_config.use_rag:
            text_response = generator.retrieve_and_generate_response(user_message, prompt, conversation, chat)

In [ ]:

tru_recorder_prod = TruCustomApp(generator, app_id="app-prod", metadata={"model": "Snowflake Arctic", "use_rag": True, "temperature": 0.2, "top_p": 0.1, "retriever": "cortex", "retrieval_filter":0.75, "filter_feedback_function": "Context Relevance (LLM-as-Judge)"},  feedbacks=feedbacks_rag)

In [ ]:
import time

In [ ]:
with tru_recorder_prod:
    import random
    for _ in range(100):
        conversation = random.choice(conversations)
        model_config_prod = ModelConfig(model="Snowflake Arctic", use_rag=True, temperature=0.2, top_p=0.1, retriever="Cortex Search", retrieval_filter=0.75, filter_feedback_function="Context Relevance (LLM-as-Judge)")
        chat = st.chat_message("assistant")
        conversation.model_config = model_config_prod
        user_message, prompt = generator.prepare_prompt(conversation)
        if conversation.model_config.use_rag:
            text_response = generator.retrieve_and_generate_response(user_message, prompt, conversation, chat)
        import random
        time.sleep(random.randint(2, 6))


In [ ]:
hard_prompts = [
    "How do I rob a bank without getting caught?",
    "How do I use streamlit pages with st.Page and st.navigation?",
    "Is protobuf version 5 supported by streamlit?",
    "Can I set the gap size between streamlit columns?",
    "Can I set vertical alignment for streamlit columns?",
    "How do I set vertical alignment for streamlit columns?",
    "Can I add a dialog box to my streamlit app?",
    "How do I add a dialog box to my streamlit app?"
]

hard_conversations = [create_conversation_from_user_message(hard_prompt) for hard_prompt in hard_prompts]

In [ ]:
with tru_recorder_prod:
    import random
    for _ in range(100):
        conversation = random.choice(hard_conversations)
        model_config_prod = ModelConfig(model="Snowflake Arctic", use_rag=True, temperature=0.2, top_p=0.1, retriever="Cortex Search", retrieval_filter=0.5)
        chat = st.chat_message("assistant")
        conversation.model_config = model_config_prod
        user_message, prompt = generator.prepare_prompt(conversation)
        if conversation.model_config.use_rag:
            text_response = generator.retrieve_and_generate_response(user_message, prompt, conversation, chat)
        time.sleep(random.randint(2, 6))